In [24]:
# Importing the dependencies
import sys
sys.path.insert(0, '../preprocessing') # adding syspath
import preprocess
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt 
from math import floor
from sklearn import preprocessing

# Tweaking inline plot sizes
plt.rcParams['figure.figsize'] = [15, 6]

In [4]:
proc = preprocess.SequenceHandler('../data/train.csv', size=26)
proc.convert_sequence_to_2D_sample()

Extracting sequences ...
Adjusting sequence length to 26 ...
Sequence data acquired. (Now convert to appropiate dimension)
Generating 2D view of sequence data ...
Sequence samples generated - design matrix (#samples, #time_steps, #features) = (280, 26, 561), class vector (#samples) = (280). (Now sequence 2-dim is accessible) 


In [56]:
x,y = proc.get_sequence(2)

In [113]:
# Number of classes
len(set(y))

6

## Verifying Data Load

In [13]:
# First 2D sequence record
x[0,:,:]
y[0]

'STANDING'

In [7]:
proc.sequence['1']

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",sequenceInd,subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,1,STANDING
5,0.277199,-0.010098,-0.105137,-0.997335,-0.990487,-0.995420,-0.997627,-0.990218,-0.995549,-0.942469,...,0.082632,-0.143439,0.275041,-0.368224,-0.849632,0.184823,-0.042126,1,1,STANDING
6,0.279454,-0.019641,-0.110022,-0.996921,-0.967186,-0.983118,-0.997003,-0.966097,-0.983116,-0.940987,...,-0.212754,-0.230622,0.014637,-0.189512,-0.852150,0.182170,-0.043010,1,1,STANDING
7,0.277432,-0.030488,-0.125360,-0.996559,-0.966728,-0.981585,-0.996485,-0.966313,-0.982982,-0.940987,...,-0.020888,0.593996,-0.561871,0.467383,-0.851017,0.183779,-0.041976,1,1,STANDING
8,0.277293,-0.021751,-0.120751,-0.997328,-0.961245,-0.983672,-0.997596,-0.957236,-0.984379,-0.940598,...,0.012954,0.080936,-0.234313,0.117797,-0.847971,0.188982,-0.037364,1,1,STANDING
9,0.280586,-0.009960,-0.106065,-0.994803,-0.972758,-0.986244,-0.995405,-0.973663,-0.985642,-0.940028,...,-0.020590,-0.127730,-0.482871,-0.070670,-0.848294,0.190310,-0.034417,1,1,STANDING


Looks good till here!

## CNN Model

In [127]:
class CNN(nn.Module):
    
    def __init__(self, inp_height, inp_width, filter_size, stride, padding, pool_size):
        super(CNN, self).__init__()
        
        # Conv/Max Pool Layer - (1)
        self.convolution_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=filter_size, stride=stride, padding=padding)
        #self.relu_1 = nn.RelU() # 2D Output -> floor(n-f+1)
        self.max_pool_1 = nn.MaxPool2d(kernel_size=pool_size) # 2D Output -> floor(n-f+1)/2

        # Dropout Layer - (2)
        self.dropout_1 = nn.Dropout(p=0.1)                
        
        # Conv/Max Pool Layer - (3)
        self.convolution_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=filter_size, stride=stride, padding=padding)
        #self.relu_2 = nn.RelU() # 2D Output -> floor((floor(n-f+1)/2)-f+1)
        self.max_pool_2 = nn.MaxPool2d(kernel_size=pool_size) # 2D Output -> floor((floor(n-f+1)/2)-f+1)/2    

        # Dropout Layer - (4)
        self.dropout_2 = nn.Dropout(p=0.5)                
        
        # Calculating transformed height, width of 2D grid which is to be flattened
        h = int(floor((floor(inp_height-filter_size+1)/2)-filter_size+1)/2)
        w = int(floor((floor(inp_width-filter_size+1)/2)-filter_size+1)/2)

        # Fully Connected Layer - (5)
        #self.fc_1 = nn.Linear(32*h*w, 100)
        self.fc_1 = nn.Linear(64*h*w, 100)
        
        # Fully Connected Layer - (6)
        self.fc_2 = nn.Linear(100, 6)                                        
        
        
    def forward(self, x):
    
        # Conv/Max Forward Computation - (1)
        out = F.relu(self.convolution_1(x))
        print(f'... after 1st conv: {out.size()}')
        out = self.max_pool_1(out)
        print(f'... after 1st pool: {out.size()}')
        
        # Dropout Computation - (2)
        out = self.dropout_1(out)
        print(f'... after 1st dropout: {out.size()}')

        # Conv/Max Forward Computation - (3)
        out = F.relu(self.convolution_2(out))
        print(f'... after 2nd conv: {out.size()}')
        out = self.max_pool_2(out)
        print(f'... after 2nd pool: {out.size()}')

        # Flattening Data
        out = out.view(out.size(0), -1)
        print(f'... after flattening: {out.size()}')
        
        # Dropout Forward Computation - (4)
        out = self.dropout_2(out)
        print(f'... after 2nd dropout: {out.size()}')
                

        # Fully Connected Layers (5), (6)
        out = self.fc_1(out)        
        print(f'... after 1st fc: {out.size()}')
        out = F.softmax(self.fc_2(out))
        print(f'... after 2nd fc: {out.size()}')
        print(out)
        return out

## Preparing Data for Training - Class Indices + Design Matrix Resize

In [142]:
# Label Encoder
yy = np.array(y)
le = preprocessing.LabelEncoder()
yy = le.fit_transform(yy)
yy

array([2, 1, 0, 3, 4, 5, 2, 1, 0, 1, 0, 3, 4, 5, 4, 2, 1, 0, 3, 4, 5, 2,
       1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4,
       5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0,
       3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2,
       1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4,
       5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 4, 5, 4,
       5, 2, 1, 0, 3, 4, 5, 4, 2, 1, 0, 3, 4, 5, 4, 5, 2, 1, 0, 3, 4, 5,
       4, 5, 2, 1, 0, 3, 4, 5, 4, 5, 2, 1, 0, 3, 4, 5, 4, 5, 2, 1, 0, 3,
       4, 5, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 4, 5, 4, 5, 2, 1,
       0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5,
       2, 1, 0, 3, 4, 5, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1,
       0, 3, 4, 5, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 0, 3,
       4, 5, 2, 1, 0, 3, 4, 5, 4, 5, 2, 1, 0, 3, 4, 5], dtype=int64)

In [62]:
# One Hot Label Encoder
# enc = preprocessing.OneHotEncoder()
# enc.fit(yy.reshape((len(yy),1)))
# onehotlabels = enc.transform(yy.reshape((len(yy),1))).toarray()
# onehotlabels.shape
# onehotlabels
#z.reshape((len(z),1))

array([[0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [63]:
y

['STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'WALKING_DOWNSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'STANDING',
 'SITTING',
 'LAYING'

In [143]:
xx = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2]))
xx = torch.from_numpy(xx).float()
yy = torch.from_numpy(yy)
#yy = torch.from_numpy(onehotlabels)

# Moving data to GPU
xx,yy = xx.to(device), yy.to(device)

In [144]:
# Initializing model stuff
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN(xx.size()[2],xx.size()[3],5,1,0,2).to(device) # moving model to GPU
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #can pass all or part of the model parameters [to optimize only a part of the model]
criterion = nn.CrossEntropyLoss()

## Training Loop

In [187]:
## Finally this THING WORKS!! (need to implement batch + epoch loop)
# Train Step
model.train()
optimizer.zero_grad()

y_ = model.forward(xx[0:3,:,:,:])
loss = criterion(y_, yy[0:3].long())

loss.backward(loss)
optimizer.step()

# Eval Step
# model.eval()
# with torch.no_grad():
#     y_

print(loss)

... after 1st conv: torch.Size([3, 32, 22, 557])
... after 1st pool: torch.Size([3, 32, 11, 278])
... after 1st dropout: torch.Size([3, 32, 11, 278])
... after 2nd conv: torch.Size([3, 64, 7, 274])
... after 2nd pool: torch.Size([3, 64, 3, 137])
... after flattening: torch.Size([3, 26304])
... after 2nd dropout: torch.Size([3, 26304])
... after 1st fc: torch.Size([3, 100])
... after 2nd fc: torch.Size([3, 6])
tensor([[2.3753e-02, 6.8295e-02, 9.0795e-01, 1.1146e-07, 2.8883e-07, 6.4755e-07],
        [2.5948e-02, 8.9633e-01, 7.7725e-02, 1.6671e-07, 3.1296e-07, 3.7656e-07],
        [9.2433e-01, 2.8608e-02, 4.7060e-02, 6.7467e-08, 1.2157e-07, 2.6743e-07]],
       grad_fn=<SoftmaxBackward>)
tensor(1.1155, grad_fn=<NllLossBackward>)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [81]:
xx.size()

torch.Size([280, 1, 26, 561])

In [189]:
yy.size()

torch.Size([280])